## Import data

In [73]:
import pandas as pd
# Read our station csv files, join them and do some date processing
station_status = pd.read_csv('station_status.csv')
station_information = pd.read_csv('station_information.csv')
stations = station_information.join(
    station_status.reset_index().set_index('station_id'), on='station_id', lsuffix='_information'
)
stations = stations.reset_index()
stations['last_reported'] = pd.to_datetime(stations['last_reported'], unit='s')
stations['last_reported'] = stations['last_reported'].dt.tz_localize('UTC').dt.tz_convert('Europe/Oslo')
stations = stations.sort_values(by=['station_id', 'last_reported'])

## Graph a random station

In [74]:
# let's pick a random station and graph its available bikes and docks
import numpy as np
import plotly.io as pio
pio.templates.default = "plotly_dark"
import plotly.express as px
import plotly.offline as py

random_station_id = stations['station_id'][np.random.randint(len(stations))]
selected_station = stations[stations['station_id'] == random_station_id]
fig = px.line(
    selected_station, x='last_reported', y=['num_bikes_available', 'num_docks_available'],
)
station_name = selected_station['name'].iloc[0]
fig.update_layout(
    title='Bysykkel station ' + station_name + ' bike and dock availability',
    xaxis_title='Date / Time', yaxis_title='Number', 
    legend_title='Metric',
)
trace_names = {'num_bikes_available': 'Available bikes', 'num_docks_available': 'Available docks'} 
fig.for_each_trace(lambda t: t.update(
        name = trace_names[t.name],
        legendgroup = trace_names[t.name],
        hovertemplate = t.hovertemplate.replace(t.name, trace_names[t.name])
    )
)
fig

## Changes in station bike availability by hour of day

In [75]:
# Graph the number of times the number of bikes at a station changes by the hour of day
stations['same_station_as_previous_row'] = stations['station_id'] == stations['station_id'].shift(-1)
stations['change_in_bikes'] = stations['same_station_as_previous_row'] * stations['num_bikes_available'] != stations['num_bikes_available'].shift(-1)
changed_by_hour = stations.groupby(stations.last_reported.dt.hour).change_in_bikes.sum()
fig = px.line(
    changed_by_hour, y='change_in_bikes', labels={'change_in_bikes': 'Changes in station bike availability', 'last_reported':'Hour of day'}
)
fig.update_layout(
    title='Changes in station bike availability',
    xaxis_title='Hour of day',
)
fig


## Absolute changes in bike numbers by hour

In [76]:
# Graph the absolute change in number of bikes at a station by the hour of day
stations.change_in_bikes = abs(stations['num_bikes_available'] - stations['num_bikes_available'].shift(-1)) * stations['station_id'] == stations['station_id'].shift(-1)
stations['change_in_bikes'] = stations['same_station_as_previous_row'] * abs(stations['num_bikes_available'] - stations['num_bikes_available'].shift(-1))

changed_by_hour = stations.groupby(stations.last_reported.dt.hour).change_in_bikes.sum()
fig = px.line(
    changed_by_hour, y='change_in_bikes', labels={'change_in_bikes': 'Number of bike availability changes', 'last_reported': 'Hour of day'}
)
fig.update_layout(title='Absolute change in availability of bikes')
fig

In [89]:
# Build an hour by hour heatmap of number of available bikes and docks 
import plotly.graph_objects as go

stations_heatmap = stations.groupby(
    [stations.name, stations.station_id, stations.last_reported.dt.hour],
).agg(
    mean_available_bikes=('num_bikes_available', 'mean'), 
    mean_available_docks=('num_docks_available', 'mean'),
    count=('last_reported', np.count_nonzero),
    start=('last_reported', 'min'),
    end=('last_reported', 'max'),
).round(0)

stations_heatmap.index.names = ['station_name', 'station_id', 'hour_of_day']

# build a graph with the first station
station = stations_heatmap.loc[[stations_heatmap.index.levels[0][0]]]
custom_data = np.r_[1:24, 0:1]
fig = go.Figure(data=[
    go.Bar(
        x=station.mean_available_bikes,
        y=station.index.get_level_values(level=2),
        name='Average available bikes',
        orientation='h',
        hovertemplate='Average number of bikes between %{y}:00 and %{customdata}:00 - %{x}<extra></extra>',
        customdata=custom_data,
    ), 
    go.Bar(
        x=station.mean_available_docks,
        y=station.index.get_level_values(level=2),
        name='Average available docks',
        hovertemplate='Average number of docks between %{y}:00 and %{customdata}:00 - %{x}<extra></extra>',
        orientation='h',
        customdata=custom_data,
    ),
])

title = 'Average dock and bike availability OsloBysykkel.no stations<br><sup>Based on {0} station statuses between {1} and {2}</sup>'.format(
    station['count'].sum(), station['start'].iloc[0].strftime('%Y-%m-%d'), station['end'].iloc[0].strftime('%Y-%m-%d')
)
title += '<br><sup>See github.com/kalli/bysykkel-data for more information.</sup>'

fig.update_layout(
    title=title,
    xaxis_title='Count',
    yaxis_title='Hour of day',
    yaxis=dict(range=[0, 23]),
    barmode='stack',
)

# Add an annotation about when the system is closed
annotation = 'The Bysykkel system is closed between 01:00 and 05:00'
center_x = (station.mean_available_bikes[0] + station.mean_available_docks[0]) / 2
fig.add_annotation(
    x=center_x, y=3, height=50, text=annotation, showarrow=False, 
    bgcolor='#000000',
)
fig['layout']['yaxis']['autorange'] = 'reversed'

fig.update_yaxes(range=[0, 23])
fig.update_yaxes(tickvals=[0, 4, 8, 12, 16, 20])

# Build a dropdown so that the user can select information for a specific station
all_station_data = []
for station in station_information.itertuples():
    station_chart = stations_heatmap.loc[station.name]
    station_data = [
        [station_chart.mean_available_bikes, station_chart.index], 
        [station_chart.mean_available_docks, station_chart.index]
    ],
    all_station_data.append(
        dict(
            args=[ {'x': [station_chart.mean_available_bikes, station_chart.mean_available_docks] } ], 
            label=station.name, 
            method='update',
        )
    )
all_station_data.sort(key=lambda x: x['label'])
fig.update_layout(
    updatemenus=[
        dict(
            buttons=all_station_data,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
        ),
    ]
)
fig.write_html('./charts/bysykkel-station-hour-by-hour-availability.html')